## 모듈

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as BS
import time
import selenium
import requests
import re
import pandas as pd
import json

## 크롤링

In [6]:
# CSRF Token
# requests.Session() -> 토큰값 고정
payload = {"pageNum" : "1",
"pageSize" : "5",
"searchShopName" : "",
"searchSido" : "11",
"searchGugun" : "",
"searchDong" : "",
"searchType" : "",
"searchTypeService" : "0",
"searchTypeToto" : "0",
"searchTypeCafe25" : "0",
"searchTypeInstant" : "0",
"searchTypeDrug" : "0",
"searchTypeSelf25" : "0",
"searchTypePost" : "0",
"searchTypeAtm" : "0",
"searchTypeWithdrawal" : "0",
"searchTypeTaxrefund" : "0",
"searchTypeSmartAtm" : "0",
"searchTypeSelfCookingUtensils" : "0",
"searchTypeDeliverySerivce" : "0",
}
post_url = "http://gs25.gsretail.com/gscvs/ko/store-services/locationList?CSRFToken={}"
url = 'http://gs25.gsretail.com/gscvs/ko/store-services/locations'
total = []
with requests.Session() as s:
    r = s.get(url)
    bs = BS(r.text)
    csrf = bs.find("form", id="CSRFForm").find("input")["value"]
    master = {x.text : x['value'] for x in bs.find("select", id="stb1").findAll("option")[1:]}
    for code in master.values():
        payload['pageSize'] = 5000
        payload['searchSido'] = code
        r2 = s.post(post_url.format(csrf), data=payload)
        total.append(pd.DataFrame(json.loads(r2.json())["results"]))


In [7]:
gs = pd.concat(total)

## 데이터 확인

In [8]:
gs.head()

,offeringService,shopCode,longs,address,shopName,lat
0,[post],VQ037,37.52831065,"서울 양천구 목동서로159-1 (목1동 917-1, CBS방송국1층)",GS25CBS점,126.87486792
1,"[cafe25, post]",VO395,37.5783899,"서울 마포구 매봉산로75, 1층 107호 (상암동 1610)",GS25DDMC점,126.89294205
2,"[cafe25, drug, post]",VER76,37.57410234,"서울 서대문구 가재울미래로2, 203동 101호 (남가좌동 385, 2단지)",GS25DMC가재울점,126.91639401
3,"[cafe25, instant, post]",V9A48,37.50235508,"서울 강남구 테헤란로223, 1층 101호 (역삼동 677-25, 큰길타워빌딩)",GS25DXLAB점,127.04069826
4,"[drug, withdrawal]",VI989,37.55513349,서울 강동구 고덕로39 (암사3동 441-14),GS25G고덕점,127.12956973


## 데이터 추출

In [9]:
gs.to_excel('./data/gs25.xlsx')